In [1]:
from IPython.display import display
import ipywidgets.widgets as widgets

from jetbot import Camera, bgr8_to_jpeg

image = widgets.Image(format='jpeg', width=224, height=224)
display(image)

Image(value=b'', format='jpeg', height='224', width='224')

In [2]:
file = open("dog.jpg", "rb")
image.value = file.read()


In [8]:
file = open("turkish_coffee.jpg", "rb")
image.value = file.read()

In [3]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('alexnet_trt.pth'))
device = torch.device('cuda')
model = model_trt.to(device)

In [ ]:
import torchvision, torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

device = torch.device('cuda')
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(bytearray(image.value))
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

x = preprocess(image)
y = model_trt(x)

In [4]:
from PIL import Image
import torch

device = torch.device('cuda')

from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])



In [5]:
with open('imagenet_classes.txt') as f:
  labels = [line.strip() for line in f.readlines()]

In [9]:
import io
import PIL.Image as Image

img=Image.open(io.BytesIO(image.value))
img_t = transform(img)
batch_t = torch.unsqueeze(img_t, 0).to(device)
y = model_trt(batch_t)


In [10]:
	_, index = torch.max(y, 1)
percentage = torch.nn.functional.softmax(y, dim=1)[0] * 100
print(labels[index[0]], percentage[index[0]].item())


963, pizza 66.68255615234375
